<a href="https://colab.research.google.com/github/seodaegal/Textmining/blob/main/3_1%EA%B0%95_%ED%85%8D%EC%8A%A4%ED%8A%B8_%EB%B6%84%EB%A5%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

3-1강. 텍스트 분류
=====
요즘IT 주제 예측
-----
지난 시간에 이어 이번에는 다중 클래스 텍스트 분류를 해보겠습니다.<br>
이번에 살펴볼 모델은 나이브 베이즈 분류기와 서포트 벡터 머신입니다.

데이터는 지난 2월 13일 기준 청와대 국민청원, 참여 인원 200명 이상(일부 199명)의 청원 16035개를 크롤링한 데이터입니다.<br>
해당 데이터의 특징은 이미 토픽이 주어져 있다는 것입니다. 이 분류에 맞게 잘 판단하는지 확인해봅시다.

In [ ]:
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os

path = '/content/drive/MyDrive/딥다이브/3강/3강. 텍스트 분류'
filename = '요즘IT_2023.04.27.csv'
df = pd.read_csv(os.path.join(path, filename))

In [ ]:
df.head()

,제목,작성자,날짜,조회수,분류,본문
0,"아마존, 네이버, 스타벅스..공룡 기업이 '커뮤니티' 하는 이유",프로덕트 세계,10시간 전,1.8K,기획,[프세 커뮤니티 리포트 #1] 요즘 핫한 기업은 왜 ‘커뮤니티’에 집중할까?\n\n...
1,﻿﻿ChatGPT로 쿠버네티스 관리하는 방법,테크유람,15시간 전,3.0K,개발,"최근 ChatGPT에 대한 관심이 뜨겁습니다. ChatGPT는 단순 개발부터 보안,..."
2,프롬프트 한 줄로 다섯 가지 컬러 만드는 ‘AI Colors’,지금 써보러 갑니다,1일 전,2.7K,디자인,챗GPT의 등장으로 서비스는 물론 업무 과정에도 많은 영향을 끼치고 있는데요. 특히...
3,데이터 주도 프로덕트 개발을 위한 실험 환경 세팅하기,요즘IT의 번역글들,1일 전,2.4K,개발,본문은 deepL을 활용해 만든 해외 번역 콘텐츠입니다. 필자인 빅터 세민(Vict...
4,클라우드 비용 최적화: 수백만 달러 절약한 쿠팡의 노력,쿠팡 엔지니어링,2일 전,7.0K,개발,국내 IT기업은 한국을 넘어 세계를 무대로 할 정도로 뛰어난 기술과 아이디어를 자랑...


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1469 entries, 0 to 1468
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   제목      1469 non-null   object
 1   작성자     1469 non-null   object
 2   날짜      1469 non-null   object
 3   조회수     1469 non-null   object
 4   분류      1468 non-null   object
 5   본문      1469 non-null   object
dtypes: object(6)
memory usage: 69.0+ KB


In [ ]:
!pip install konlpy

from konlpy.tag import Okt
okt = Okt()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 45.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 31.7 MB/s eta 0:00:00


In [ ]:
df['본문_전처리'] = df['본문'].map(okt.nouns)

In [ ]:
df['본문_전처리'] = df['본문_전처리'].map(' '.join)

In [ ]:
# 결측치 확인
df[df['본문_전처리'].isnull()]

,제목,작성자,날짜,조회수,분류,본문,본문_전처리


In [ ]:
df.dropna(inplace = True)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB # 다중 Naive Bayes 분류기
from sklearn.metrics import accuracy_score

## Naive Bayes

In [ ]:
# 데이터 스플릿(훈련 데이터와 테스트 데이터)
train_texts, test_texts, train_labels, test_labels = train_test_split(df['본문_전처리'], df['분류'], test_size=0.3, random_state=0)

In [ ]:
# TF-IDF 벡터화
vectorizer = TfidfVectorizer(min_df = 10, ngram_range = (1,2))
train_features = vectorizer.fit_transform(train_texts)
print(train_features)

  (0, 4195)	0.035978989051695394
  (0, 4844)	0.03507188170971841
  (0, 1472)	0.02884732257098691
  (0, 6817)	0.03702616762666347
  (0, 4057)	0.032040157862111854
  (0, 4704)	0.032317470411735244
  (0, 1948)	0.032908557326590995
  (0, 168)	0.035978989051695394
  (0, 1587)	0.022226930262770932
  (0, 6539)	0.03761725454151922
  (0, 4194)	0.032317470411735244
  (0, 2982)	0.03103190898749986
  (0, 3695)	0.031270291836767175
  (0, 1591)	0.024949373435306735
  (0, 4265)	0.03507188170971841
  (0, 1119)	0.03427175632868031
  (0, 1538)	0.03648242037461056
  (0, 4455)	0.03648242037461056
  (0, 4594)	0.024854361898823868
  (0, 4476)	0.03648242037461056
  (0, 5705)	0.032040157862111854
  (0, 1851)	0.032908557326590995
  (0, 2072)	0.03507188170971841
  (0, 4253)	0.07014376341943682
  (0, 6325)	0.06354744631936467
  :	:
  (1026, 2945)	0.006528704794484674
  (1026, 4250)	0.022789041935290712
  (1026, 2493)	0.010919988019482196
  (1026, 52)	0.004805104627951748
  (1026, 4574)	0.036429398382575084
  (10

In [ ]:
# Naive Bayes 모델
nb = MultinomialNB(alpha = 0.01) # alpha는 hyper parameter로 사람이 직접 지정해줘야 합니다.

In [ ]:
# Naive Bayes 모델 훈련 데이터로 학습
nb.fit(train_features, train_labels)

MultinomialNB(alpha=0.01)

In [ ]:
# 테스트 데이터 벡터화, transform = 알 수 없는 값, fit transform이 아닌 transform (학습에서만 fittransform)
test_features = vectorizer.transform(test_texts)

In [ ]:
pd.DataFrame(test_features.todense()) #새로운 단어는 사용 못함 (포함이 안되어있다)

,0,1,2,3,4,5,6,7,8,9,...,7122,7123,7124,7125,7126,7127,7128,7129,7130,7131
0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.015994,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0
1,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.046714,0.0,0.0,0.000000,0.0,0.0
2,0.0,0.0,0.043867,0.0,0.043148,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0
3,0.0,0.0,0.074984,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0
4,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
436,0.0,0.0,0.017788,0.0,0.034993,0.0,0.0,0.000000,0.0,0.027491,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0
437,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0
438,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0
439,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0


In [ ]:
# Naive Bayes 모델로 분류 예측하기
nb.predict(test_features)

array(['프로덕트', '프로덕트', '프로덕트', '기획', '개발', '기획', '디자인', '디자인', '개발', '기획',
       '프로덕트', '개발', '프로덕트', '개발', '비즈니스', '기획', '프로덕트', '프로덕트', '비즈니스',
       '아웃소싱', '기획', '개발', '기획', '기획', '기획', '기획', '프로덕트', '기획', '디자인',
       '개발', '기획', '디자인', '개발', '기획', '개발', '디자인', '디자인', '기획', '개발',
       '개발', '디자인', '디자인', '개발', '개발', '기획', '비즈니스', '개발', '개발', '디자인',
       '비즈니스', '기획', '개발', '개발', '기획', '개발', '프로덕트', '프로덕트', '개발', '프로덕트',
       '프로덕트', '프로덕트', '개발', '개발', '프로덕트', '아웃소싱', '프로덕트', '개발', '프로덕트',
       '기획', '기획', '프로덕트', '개발', '기획', '디자인', '기획', '디자인', '기획', '기획',
       '프로덕트', '기획', '디자인', '프로덕트', '기획', '프로덕트', '아웃소싱', '기획', '개발',
       '프로덕트', '기획', '기획', '프로덕트', '개발', '프로덕트', '개발', '개발', '프로덕트', '기획',
       '프로덕트', '프로덕트', '기획', '개발', '비즈니스', '프로덕트', '디자인', '기획', '개발',
       '개발', '기획', '기획', '기획', '개발', '프로덕트', '디자인', '디자인', '디자인', '비즈니스',
       '기획', '기획', '기획', '프로덕트', '개발', '프로덕트', '개발', '개발', '기획', '기획',
       '기획', '디자인', '프로덕트', '개발', '기획', '기획', '디자인', '개발', 

In [ ]:
# 분류 예측 결과 확인
pred_labels = nb.predict(test_features)
print('잘못 분류된 아티클: {} out of {}'.format((pred_labels != test_labels).sum(),len(test_labels)))
print('정확도: %.2f' % accuracy_score(test_labels, pred_labels)) #73은 높은편이 아니다ㅠ

잘못 분류된 아티클: 117 out of 441
정확도: 0.73


In [ ]:
test_df = pd.DataFrame({'본문_전처리' : test_texts, '분류_예측' : pred_labels}); test_df

,본문_전처리,분류_예측
362,지난 손흥민 선수 소속 프리미어리그 축구 구단 토트넘 내한 경기 화제 쿠팡 플레이 ...,프로덕트
975,몇 해전 모두 위 서비스 것 집중 뉘앙스 유니버설 디자인 과 말 보편성 추구 등장 ...,프로덕트
980,신세계 백화점 이마트 트 레이더스 스타 필드 심지어 스타벅스 신세계 의 이커머스 게...,프로덕트
1046,시리즈 구성 글 이번 글 모델 개념 구성 대해 다음 편 모델 수립 제품 전략 실무 ...,기획
785,해외 유명 기업 각자 신들 블로그 운영 해 개발 방법 기업 문화 등 소개 요즘 기업...,개발
...,...,...
1025,세계 분야 혁신 스마트폰 등장 수 중반 스마트폰 등장 후 데스크톱 의 시대 본격 스...,프로덕트
339,일자리 개념 차 산업혁명 코로나 의 여파 취업 시장 고용 형태 변화 실제 세대 직장...,프리랜싱
364,국내 유명 기업 한국 세계 무대 정도 기술 아이디어 자랑 이 기업 블로그 통해 정보...,개발
705,해외 유명 기업 각자 자신 블로그 운영 개발 방법 기업 문화 등 소개 요즘 기업 블...,기획


In [ ]:
test_df = df[['제목', '작성자', '분류', '본문', '본문_전처리']].merge(test_df, on = ['본문_전처리']).copy()

In [ ]:
test_df[test_df['분류_예측'] != test_df['분류']] #분류가 다른 것만

,제목,작성자,분류,본문,본문_전처리,분류_예측
3,당신이 성장하지 못하는 이유,유영모,비즈니스,"나는 지난 몇 년간 개발자를 코칭하며 ‘프론트냐 백엔드냐’를 고민하는 당신에게, 개...",나 지난 몇 년 개발자 코칭 프론트 백엔드 를 고민 당신 개발자 코칭 가지 장애 우...,기획
8,구글‧애플‧테슬라‧현대차 뛰어든 모빌리티 UX/UI의 미래,디자인민수,디자인,UX 디자이너가 바라본 자동차 UX/UI와 모빌리티 시장의 미래\n어릴 적 미술시간...,디자이너 자동차 모빌리티 시장 미래 적 미술 시간 과학 상상 그리기 기억 아이 주로...,개발
13,"‘버티컬 커머스’의 강자 컬리, 29CM, 오늘의집 주문서 뜯어보기",POLEE,기획,이커머스 주문서 뜯어보기 2편\n\n현재 내가 속한 팀은 매주 주요 지표의 증감을 ...,이커머스 주문 보기 편 현재 내 속 팀 주요 지표 증감 확인 액션 아이템 도출 최근...,프로덕트
15,"드디어 만난 '애플페이', 애플워치로 직접 써봤습니다",큰그림기획,프로덕트,이번 글은 3/21일 국내 서비스를 시작한 애플페이에 대한 리뷰 콘텐츠입니다. 혹시...,이번 글 국내 서비스 시작 애플 페이 대한 리뷰 콘텐츠 혹시 애플 페이 직접 보신 ...,비즈니스
19,AI 프롬프트 엔지니어링과 프롬프트를 훔치는 '인젝션',muralmural,개발,최근 대화형 인공지능에 대한 반응이 뜨겁습니다. ChatGPT를 필두로 인공지능의 ...,최근 대화 인공 지능 대한 반응 를 필 로 인공 지능 시대 본격 시작 것 이번 글 ...,기획
...,...,...,...,...,...,...
412,"소수를 고려해야만 하는 디자인, 3편",WHTM,디자인,"디지털 키오스크는 어쩌면 자율주행 기술,매장 무인화에 대한 두려움의 예고편일지도 모...",디지털 키 오스 어쩌면 자율 주행 기술 매장 무인 대한 두려움 예고편 일지 자율 주...,프로덕트
415,"기획 용어 1부: 웹, 앱 UI 설계 전 알아야 하는 기본 개념",NINA.C,기획,"첫 번째,화면 및 팝업 관련 기본 용어 알기\n\n구글링을 해도 명확하게 뜻이 나오...",첫 화면 및 팝업 관련 기본 용어 구글링 해도 뜻 선배 동료 용어 실제 프로젝트 진...,디자인
419,온라인 쇼핑몰 창업 A-Z까지: 나만의 인터넷 쇼핑몰을 구축하라,IT 모니카,기획,우리는 5G인터넷 플랫폼 시대에 살고 있습니다. 5G가 발달하면서 인터넷의 무선 서...,우리 인터넷 플랫폼 시대 살 발달 인터넷 무선 서비스 속도 모바일 환경 예전 광고 ...,프로덕트
435,내가 엔지니어링 매니저로 일하면서 저지른 실수들,요즘IT의 번역글들,개발,"본문은 위시켓과 번역가 전리오가 함께 만든 해외 콘텐츠 기반 번역문입니다.웹디자인,...",본문 위시켓 번역가 리오 해외 콘텐츠 기반 번역 문 웹디자인 대한 정보 발행 글 작...,기획


In [ ]:
def tokenizer(text):
    word_list = okt.pos(text)
    pos_list = ['Noun', 'Alpha']
    pos_filtered_word_list = [word[0] for word in word_list if word[1] in pos_list]

    return pos_filtered_word_list

In [ ]:
train_texts, test_texts, train_labels, test_labels = train_test_split(df['본문'], df['분류'], test_size=0.3, random_state=0)

# TF-IDF 벡터화
vectorizer = TfidfVectorizer(min_df = 10, ngram_range = (1,2), tokenizer = tokenizer)
train_features = vectorizer.fit_transform(train_texts)
print(train_features)

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


  (0, 182)	0.03234830995572846
  (0, 164)	0.031097610537544427
  (0, 1244)	0.02692249289370034
  (0, 4278)	0.026382721088604018
  (0, 1081)	0.031097610537544427
  (0, 1064)	0.024868948353034968
  (0, 10187)	0.02711372868635726
  (0, 6486)	0.03335454712427609
  (0, 1128)	0.02377074876430025
  (0, 7448)	0.016586014564617876
  (0, 978)	0.029459639143713384
  (0, 5482)	0.010269265082227145
  (0, 5267)	0.02975352286853109
  (0, 7439)	0.031097610537544427
  (0, 1163)	0.026382721088604018
  (0, 8240)	0.05891927828742677
  (0, 5692)	0.016485342223648176
  (0, 6816)	0.03283044092364046
  (0, 2637)	0.02573090505413632
  (0, 1195)	0.02323735432509337
  (0, 6545)	0.026557325242015317
  (0, 9903)	0.03283044092364046
  (0, 8605)	0.029179429480431996
  (0, 6276)	0.028409435199517743
  (0, 295)	0.03148635325462711
  :	:
  (1026, 5335)	0.013165983034718989
  (1026, 959)	0.08599202874875383
  (1026, 7009)	0.026438699088490433
  (1026, 8517)	0.004662362513381515
  (1026, 3244)	0.009173635173621577
  (102

In [ ]:
# Naive Bayes 모델
nb = MultinomialNB(alpha = 0.01)

# Naive Bayes 모델 훈련 데이터로 학습
nb.fit(train_features, train_labels)

# 테스트 데이터 벡터화
test_features = vectorizer.transform(test_texts)

# 분류 예측 결과 확인
pred_labels = nb.predict(test_features)
print('잘못 분류된 아티클: {} out of {}'.format((pred_labels != test_labels).sum(),len(test_labels)))
print('정확도: %.2f' % accuracy_score(test_labels, pred_labels))
#정확도가 늘었다!! 0.02...

잘못 분류된 아티클: 112 out of 441
정확도: 0.75
